In [1]:
import re, os, sys
from functools import reduce
from itertools import combinations as comb, permutations as perm, combinations_with_replacement as combr
from operator import itemgetter
from pprint import pprint
from math import *
from collections import defaultdict
import networkx as nx
import numpy as np
from dataclasses import dataclass, field

def dprint(d):
    pprint(dict(d))
def lmap(*args, **kwargs):
    return list(map(*args, **kwargs))
def lmapi(*args, **kwargs):
    return list(map(int, *args, **kwargs))
def enum(*args, **kwargs):
    return enumerate(*args, **kwargs)

In [4]:
input_fname = "../inputs/day15-demo.in"

In [32]:
input_fname = "../inputs/day15.in"

In [33]:
def reread():
    lines = []
    with open(input_fname) as ifile:
        def r(): return ifile.readline().strip()
        def rs(): return ifile.readline().strip().split()
        def ri(): return int(ifile.readline().strip())
        def ris(): return list(map(int, ifile.readline().strip().split()))

        minx = miny = 1000
        maxx = maxy = 0
        for line in ifile:
            line = line.replace("Sensor at x=", "") \
                       .replace(", y=", " ") \
                       .replace(": closest beacon is at x=", " ")
            sensorx, sensory, beaconx, beacony = lmapi(line.split())
            minx = min(minx, sensorx, beaconx)
            maxx = max(maxx, sensorx, beaconx)
            miny = min(miny, sensory, beacony)
            maxy = max(maxy, sensory, beacony)
            lines.append(((sensorx, sensory), (beaconx, beacony)))

    return minx, maxx, miny, maxy, lines

minx, maxx, miny, maxy, lines = reread()
lines[:10]

[((1638847, 3775370), (2498385, 3565515)),
 ((3654046, 17188), (3628729, 113719)),
 ((3255262, 2496809), (3266439, 2494761)),
 ((3743681, 1144821), (3628729, 113719)),
 ((801506, 2605771), (1043356, 2000000)),
 ((2933878, 5850), (3628729, 113719)),
 ((3833210, 12449), (3628729, 113719)),
 ((2604874, 3991135), (2498385, 3565515)),
 ((1287765, 1415912), (1043356, 2000000)),
 ((3111474, 3680987), (2498385, 3565515))]

In [34]:
from tqdm import trange

## Part 1

In [5]:
minx, maxx, miny, maxy

(-645204, 3983949, 1000, 4169166)

In [32]:
# ar = np.zeros((maxx-minx+1, maxy-miny+1), dtype=int)
intervals = []
Y = 10
Y = 2000000
for (sx, sy), (bx, by) in lines:
    d = abs(sx - bx) + abs(sy - by)
    coverw = d - abs(sy - Y)
    if coverw >= 0:
        intervals.append((sx-coverw, sx+coverw+1))
minintx = min(map(itemgetter(0), intervals))
maxintx = max(map(itemgetter(1), intervals))

# print(minintx, intervals)
ar = np.zeros(maxintx-minintx+1, dtype=int)
for l, r in intervals:
    ar[l-minintx: r-minintx] = 1
for _, (bx, by) in lines:
    if by == Y:
        ar[bx] = 0
ar.sum()

4724228

## Part 2

### Attempt 1
(naive)

In [83]:
maxdim = 20 + 1
maxdim = 4000000 + 1
ar = np.zeros(maxdim, dtype=int)
ds = [abs(sx - bx) + abs(sy - by) for (sx, sy), (bx, by) in lines]
for Y in trange(maxdim):
    ar.fill(1)
    for ((sx, sy), _), d in zip(lines,ds):
        # d = abs(sx - bx) + abs(sy - by)
        coverw = d - abs(sy - Y)
        if coverw >= 0:
            # print("found", Y, coverw, sx-coverw, sx+coverw+1)
            ar[max(sx-coverw,0): sx+coverw+1] = 0
    if np.count_nonzero(ar):
        miss = list(zip(*np.nonzero(ar)))
        missx = miss[0][0]
        print(missx, Y, missx*4000000 + Y)
        break
# print(miss)
# missy, missx = miss[0]
# print(missx * 4000000 + missy)


  0%|          | 775/4000001 [00:09<13:26:30, 82.64it/s]


KeyboardInterrupt: 

In [87]:
maxdim = 20 + 1
# maxdim = 4000000 + 1
ar = np.zeros((maxdim, maxdim), dtype=int)
ds = [abs(sx - bx) + abs(sy - by) for (sx, sy), (bx, by) in lines]
for Y in trange(maxdim):
    # ar.fill(1)
    for ((sx, sy), _), d in zip(lines,ds):
        # d = abs(sx - bx) + abs(sy - by)
        coverw = d - abs(sy - Y)
        if coverw >= 0:
            # print("found", Y, coverw, sx-coverw, sx+coverw+1)
            ar[Y,max(sx-coverw,0): sx+coverw+1] += 1
    # if np.count_nonzero(ar):
    #     miss = list(zip(*np.nonzero(ar)))
    #     missx = miss[0][0]
    #     print(missx, Y, missx*4000000 + Y)
    #     break
# print(miss)
# missy, missx = miss[0]
# print(missx * 4000000 + missy)
ar


100%|██████████| 21/21 [00:00<00:00, 12750.49it/s]


array([[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 3, 2, 3, 1, 1, 1, 1],
       [1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 4, 2, 2, 1, 1, 1],
       [1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 4, 1, 2, 2, 2, 1, 1],
       [1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 3, 2, 2, 1, 2, 2, 2, 1],
       [2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 3],
       [2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 3, 3],
       [3, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 3],
       [1, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1],
       [1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1],
       [1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 2, 2, 1, 1, 1],
       [1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 3, 1, 2, 1, 1, 1, 1],
       [2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 3, 3, 3, 1, 1, 1,

In [88]:
from matplotlib import pyplot as plt

In [93]:
for row in ar:
    print(" ".join(map(str,row)))
    print()

1 1 1 1 1 1 2 2 2 2 2 1 2 1 2 1 1 1 1 1 1
1 1 1 1 1 2 2 2 2 2 2 3 1 2 2 2 1 1 1 1 1
1 1 1 1 2 2 2 2 2 2 3 2 2 1 3 2 3 1 1 1 1
1 1 1 2 2 2 2 2 2 2 1 2 2 3 2 4 2 2 1 1 1
1 1 2 2 2 2 2 2 2 1 1 1 2 2 4 1 2 2 2 1 1
1 2 2 2 2 2 2 2 1 1 1 1 1 3 2 2 1 2 2 2 1
2 2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 1 2 2 3
2 2 2 2 2 2 1 1 1 1 1 2 2 2 2 2 2 2 1 3 3
3 2 2 2 2 1 1 1 1 1 1 1 2 2 2 2 2 1 2 2 3
1 3 2 2 1 1 1 1 1 1 1 1 1 2 2 2 1 2 2 2 1
1 1 3 1 1 1 1 1 1 1 1 1 2 1 2 1 2 2 2 1 1
1 1 2 2 1 1 1 1 1 1 1 2 2 2 0 2 2 2 1 1 1
1 2 2 1 1 1 1 1 1 1 2 2 2 1 3 1 2 1 1 1 1
2 2 1 1 1 1 1 1 1 2 2 2 1 3 3 3 1 1 1 1 1
2 1 1 1 1 1 1 1 2 2 2 1 3 3 3 3 3 2 1 1 1
1 1 1 1 1 1 1 1 1 3 1 2 2 3 3 3 3 3 2 1 1
1 1 1 1 1 1 1 1 2 1 4 2 2 2 3 3 3 3 3 2 1
1 1 1 1 1 1 1 1 1 3 2 3 2 2 2 3 3 3 3 3 2
1 1 1 1 1 1 1 1 2 2 2 2 3 2 2 2 3 3 3 2 2
1 1 1 1 1 1 1 2 2 1 1 2 3 3 2 2 2 3 2 2 2
1 1 1 1 1 1 2 2 1 1 1 2 3 3 3 2 2 1 2 2 2


### Attempt 2
(with intersection of regions and maintaining just remaining valid region)

In [ ]:
@dataclass
class Rect:
    topleft: complex
    width: int
    height: int

    @property
    def botright:
        return topleft + width + height*1j

    def inside(pt: complex):
        return (topleft.real <= pt.real < botright.real) \
            and (topleft.imag <= pt.imag < botright.imag)

    def split(pt: complex, radius: int):
        y, x = pt.imag, pt.real
        top, left = topleft.imag, topleft.real
        bot, right = botright.imag, botright.real
        if self.inside(pt):  # split into 4 rects and rerun split
        elif 
        else:  # split into 2 tris and boxes
            if abs(top - y) <= radius:
                if abs(left - x) <= 
             

class Tri:
    rightpt: complex
    point1: complex
    point2: complex

In [ ]:
dim = 4000000
regions = [(0,0,)]
maxdim = 20 + 1
maxdim = 4000000 + 1
ar = np.zeros(maxdim, dtype=int)
ds = 
for Y in trange(maxdim):
    ar.fill(1)
    for ((sx, sy), _), d in zip(lines,ds):
        # d = abs(sx - bx) + abs(sy - by)
        coverw = d - abs(sy - Y)
        if coverw >= 0:
            # print("found", Y, coverw, sx-coverw, sx+coverw+1)
            ar[max(sx-coverw,0): sx+coverw+1] = 0
    if np.count_nonzero(ar):
        miss = list(zip(*np.nonzero(ar)))
        missx = miss[0][0]
        print(missx, Y, missx*4000000 + Y)
        break
# print(miss)
# missy, missx = miss[0]
# print(missx * 4000000 + missy)

### Attempt 3

(modulo region 2d partitioning)

In [7]:
from itertools import pairwise

In [35]:
# DIM = 20
DIM = 4000000
dists = {(sx, sy): abs(sx - bx) + abs(sy - by) for (sx, sy), (bx, by) in lines}

In [36]:
def make_regions(coords):
    prev = 0
    inc = [prev]
    for coord in sorted(coords):
        if coord == prev:
            continue
        prev = coord
        inc.append(prev)
    if prev != DIM:
        inc.append(prev)
    return inc

In [37]:
xinc = make_regions(map(itemgetter(0),map(itemgetter(0), lines)))
xregs = list(pairwise(xinc))
yinc = make_regions(map(itemgetter(1),map(itemgetter(0), lines)))
yregs = list(pairwise(yinc))
# xregs, yregs

In [41]:
def process_region(xlo, xhi, ylo, yhi):
    sumhi = xhi + yhi + 1
    diffhi = xhi - ylo + 1
    sumlo = xlo + ylo - 1
    difflo = xlo - yhi - 1
    for (xi, yi), di in dists.items():
        if xi <= xlo:
            if yi <= ylo: # xsign, ysign
                sumlo = max(sumlo, xi + yi + di)
            else:  # xsign, not ysign
                difflo = max(difflo, xi - yi + di)
        else:
            if yi <= ylo: # not xsign, ysign
                diffhi = min(diffhi, xi - yi - di)
            else:  # not xsign, not ysign
                sumhi = min(sumhi, xi + yi - di)
    if sumlo < sumhi - 1 and difflo < diffhi - 1:
        # print(f"for {xlo}<=x<={xhi}, {ylo}<=y<={yhi}:",
            # f"\t{sumlo} < sum < {sumhi}",
            # f"\t{difflo} < diff < {diffhi}")
        xhi = (sumhi + diffhi)/2
        xlo = (sumlo + difflo)/2
        yhi = (sumhi - diffhi)/2
        ylo = (sumlo - difflo)/2
        print(f"guess: {xlo}<x<{xhi}, {ylo}<y<{yhi}") 
        # print("##possible region", xlo, xhi, ylo, yhi)
        return True

In [42]:
for xlo, xhi in xregs:
    for ylo, yhi in yregs:
        res = process_region(xlo, xhi, ylo, yhi)
        # if res: break
    # if res: break

guess: -335387.5<x<-32922.0, 3296176.5<y<3577599.0
guess: -334882.5<x<-32922.0, 3296176.5<y<3577599.0
guess: 184324.5<x<246072.5, 4092974.5<y<4057173.5
guess: 184324.5<x<246072.5, 4092974.5<y<4057173.5
guess: 3405561.0<x<3405563.0, 3246513.0<y<3246513.0
guess: 3405561.0<x<3405563.0, 3246513.0<y<3246513.0
guess: 3405561.0<x<3405563.0, 3246513.0<y<3246513.0
guess: 3405561.0<x<3405563.0, 3246513.0<y<3246513.0
guess: 3405561.0<x<3405563.0, 3246513.0<y<3246513.0
guess: 4063958.0<x<4168597.5, 3337736.0<y<3296176.5
guess: 4063958.0<x<4319336.5, 3337736.0<y<3296176.5
guess: 4063958.0<x<4319336.5, 3337736.0<y<3296176.5


In [46]:
# visually picking out the most reasonable guess
# 3405561.0<x<3405563.0, 3246513.0<y<3246513.0
x=3405562
y=3246513

In [47]:
x*DIM + y

13622251246513